In [175]:
# import libraries
from bs4 import BeautifulSoup
import requests

url  = 'https://ph.hotels.com/search.do?q-destination=Philippines&destination-id=10233139&as-shown=false&q-localised-check-in=06%2F04%2F2021&q-check-in=2021-04-06&q-localised-check-out=07%2F04%2F2021&q-check-out=2021-04-07&q-rooms=1&q-room-0-adults=2&q-room-0-children=0&intlid=modular%20country%20landing%20%3A%3A%20SQM'
page = requests.get(url)             # connect to we'bsite
#soup = BeautifulSoup(page.contents, 'html.parser')   # vanilla call
soup = BeautifulSoup(page.text)                # use instead if returning different html formats, which is in my case

In [207]:
# hotel_cards :: BeautifulSoup Object per hotel card in site, with tag li
hotel_cardsBS = soup.select_one('body div#root ._24UiGk ._3umLRC ._3xUi2p ._3N2dPv ._1myav2 .RO5S5p ul._3f26d2')   # BeautifulSoup object of containing all cards

type(hotel_cards)

bs4.element.ResultSet

In [206]:
hotel_cards = hotel_cardsBS.find_all('li') # creates one BS obj per card from hotel card BeautifulSoup
print('we have {0} distinct hotels'.format(len(hotel_cards)))

we have 78 distinct hotels


<h3> <b> Inspecting one of the hotel cards </b> </h3>
Upon inspection the ff were idd.
<li> an a tag with further data given by the attribute href
<li> address: in _2oHhXM contains address with format: specific, town/city, zip, province, Philippines 
<li> amenities: in ?
<li> price: in ? #rate is one day for 2 persons
<li> h ref has it all, must go to href for number of rooms
<blockquote>
    <li> number of rooms in:
    <li> customer reviews in:
    <li> add landmarks, and getting around: in 
</blockquote>